In [1]:
from pathlib import Path
import subprocess

In [2]:
# run directory
# run_dir = Path('/home/gsnearing/projects/lstm_based_hydrology/extreme_year_runs/')
config_dir = Path('/home/gsnearing/projects/lstm_based_hydrology/extreme_year_configs/')
out_dir = Path('./results/')

In [3]:
# load config files
config_files = list(config_dir.glob('*.yml'))
print(f'There are {len(config_files)} experiments.')

There are 192 experiments.


In [4]:
# optimizer hypers
max_model_runs = 1e4 # 1e3 # 1e5
dds_trials = 1
use_cores_frac = 1
algorithm = 'DDS'

In [5]:
# loop over experiments
for f, config_file in enumerate(config_files):
        
    # read basefile
    with open('run_job.slurm', 'r') as file:
        filedata = file.read()

    # replace experiment name
    filedata = filedata.replace('dummy', f'{f}SAC')
 
    # replace experiment name
    filedata = filedata.replace("${1}", str(config_file))
 
    # replace experiment name
    filedata = filedata.replace("${2}", str(int(max_model_runs)))
 
    # replace algorithm
    filedata = filedata.replace("${3}", algorithm)
 
    # write to new config file
    conf = str(config_file).split('/')[-1].split('.')[0]
    job_file = Path(f"job_scripts/{conf}.slurm")
    with open(job_file, 'w') as file:
        file.write(filedata)


In [6]:
# submit all these jobs
job_files = list(Path('job_scripts').glob('**/*.slurm'))
print(f'There are {len(job_files)} experiments.')    

There are 192 experiments.


In [7]:
for file in job_files:
    run_cmd = f"sbatch {file}"
    log_file = f"log_files/{str(file).split('/')[-1].split('.')[0]}"
    with open(Path(log_file), 'w') as f:
        subprocess.Popen(run_cmd, stderr=subprocess.STDOUT, stdout=f, shell=True)